# Analysis of Robot reports from OSM Jenkins (Step 2)

## Data analysis and reports

In [ ]:
import os
import pandas as pd
#import numpy as np
#import jenkins
#import getpass
#from jenkins_lib import *
#from robot_lib import *
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt

## 0. Input parameters

In [ ]:
inputs_folder = 'etl_outputs'
outputs_folder = 'report_outputs'
database_uri = f'sqlite:///{inputs_folder}/test_executions.db'
table_known_builds = 'builds_info'
table_robot_reports = 'robot_reports'
table_robot_reports_extended = 'robot_reports_extended'

too_old_builds = "2020-12-15"
job_name = 'osm-stage_3-merge/v9.0'
#job_name = 'osm-stage_3-merge/master'

Today:

In [ ]:
today = pd.to_datetime("today").strftime('%Y-%m-%d')
today

## 1. Information about the latest builds of relevant jobs

In [ ]:
engine = create_engine(database_uri)

Finds latest builds:

In [ ]:
query_latest_builds = f'''
SELECT main.*
FROM {table_known_builds} AS main
INNER JOIN (
	SELECT job, MAX(timestamp) as ts
	FROM {table_known_builds}
	WHERE timestamp>DATETIME("{too_old_builds}")
	GROUP BY job
) AS latest_build
ON main.job=latest_build.job AND main.timestamp=ts
'''

with engine.begin() as conn:
    df_latest_builds = pd.read_sql(query_latest_builds, con=conn)

df_latest_builds['timestamp'] = pd.to_datetime(df_latest_builds.timestamp)

df_latest_builds

Retrieves reports from latest builds:

In [ ]:
table = table_robot_reports

query_robot_reports = f'''
SELECT details.*
FROM {table} AS details
INNER JOIN {table_known_builds} AS main
ON details.job=main.job AND details.build=main.build
INNER JOIN (
	SELECT job, MAX(timestamp) as ts
	FROM {table_known_builds}
	WHERE timestamp>DATETIME("{too_old_builds}")
	GROUP BY job
) AS latest_build
ON main.job=latest_build.job AND main.timestamp=ts
'''

with engine.begin() as conn:
    df_robot_reports = pd.read_sql(query_robot_reports, con=conn)

df_robot_reports['starttime'] = pd.to_datetime(df_robot_reports.starttime)
df_robot_reports['endtime'] = pd.to_datetime(df_robot_reports.endtime)

df_robot_reports.tail()

Retrieves extended reports from latest builds:

In [ ]:
table = table_robot_reports_extended

query_robot_reports = f'''
SELECT details.*
FROM {table} AS details
INNER JOIN {table_known_builds} AS main
ON details.job=main.job AND details.build=main.build
INNER JOIN (
	SELECT job, MAX(timestamp) as ts
	FROM {table_known_builds}
	WHERE timestamp>DATETIME("{too_old_builds}")
	GROUP BY job
) AS latest_build
ON main.job=latest_build.job AND main.timestamp=ts
'''

with engine.begin() as conn:
    df_robot_reports_extended = pd.read_sql(query_robot_reports, con=conn)

df_robot_reports_extended['starttime'] = pd.to_datetime(df_robot_reports_extended.starttime)
df_robot_reports_extended['endtime'] = pd.to_datetime(df_robot_reports_extended.endtime)
df_robot_reports_extended.tail()

### 1.1 Details of `v9.0` job

In [ ]:
job_name = 'osm-stage_3-merge/v9.0'

Latest `v9.0` build:

In [ ]:
df_latest_builds[df_latest_builds.job==job_name]

Failed tests (if they exist):

In [ ]:
fails_latest_v9 = df_robot_reports[(df_robot_reports.job==job_name) & (df_robot_reports.status=='FAIL')].drop(columns=['job', 'build', 'source'])
fails_latest_v9

Details of failed tests (if they exist):

In [ ]:
detailed_fails_latest_v9 = df_robot_reports_extended[(df_robot_reports_extended.job==job_name) & (df_robot_reports_extended.suite_id.isin(fails_latest_v9["id"]))].drop(columns=['job', 'build'])
detailed_fails_latest_v9['duration'] = detailed_fails_latest_v9.endtime - detailed_fails_latest_v9.starttime
detailed_fails_latest_v9

### 1.2 Details of `master` job

In [ ]:
job_name = 'osm-stage_3-merge/master'

Latest `master` build:

In [ ]:
df_latest_builds[df_latest_builds.job==job_name]

Failed tests (if they exist):

In [ ]:
fails_latest_master = df_robot_reports[(df_robot_reports.job==job_name) & (df_robot_reports.status=='FAIL')].drop(columns=['job', 'build', 'source'])
fails_latest_master

Details of failed tests (if they exist):

In [ ]:
detailed_fails_latest_master = df_robot_reports_extended[(df_robot_reports_extended.job==job_name) & (df_robot_reports_extended.suite_id.isin(fails_latest_master["id"]))].drop(columns=['job', 'build'])
detailed_fails_latest_master['duration'] = detailed_fails_latest_master.endtime - detailed_fails_latest_master.starttime
detailed_fails_latest_master

### 1.3 Details of `v10.0` job

In [ ]:
job_name = 'osm-stage_3-merge/v10.0'

Latest `v10.0` build:

In [ ]:
df_latest_builds[df_latest_builds.job==job_name]

Failed tests (if they exist):

In [ ]:
fails_latest_master = df_robot_reports[(df_robot_reports.job==job_name) & (df_robot_reports.status=='FAIL')].drop(columns=['job', 'build', 'source'])
fails_latest_master

Details of failed tests (if they exist):

In [ ]:
detailed_fails_latest_master = df_robot_reports_extended[(df_robot_reports_extended.job==job_name) & (df_robot_reports_extended.suite_id.isin(fails_latest_master["id"]))].drop(columns=['job', 'build'])
detailed_fails_latest_master['duration'] = detailed_fails_latest_master.endtime - detailed_fails_latest_master.starttime
detailed_fails_latest_master

## 2. Retrieves all currrent data for aggregate analytics

In [ ]:
too_old_builds

In [ ]:
#query_known_builds = f'SELECT * FROM {table_known_builds} WHERE job="{job_name}" AND timestamp>{too_old_builds}'
query_known_builds = f'SELECT * FROM {table_known_builds} WHERE timestamp>DATETIME("{too_old_builds}")'

with engine.begin() as conn:
    df_known_builds = pd.read_sql(query_known_builds, con=conn)

# Fixes some special data types
df_known_builds['timestamp'] = pd.to_datetime(df_known_builds.timestamp)
df_known_builds['job'] = df_known_builds.job.astype('category')
df_known_builds['duration'] = pd.to_timedelta(df_known_builds.duration.astype('float')*1000, unit='us')
df_known_builds['build_result'] = df_known_builds.build_result.astype('category')
df_known_builds['test_result'] = df_known_builds.test_result.astype('category')

# Keeps only timestamps newer than the "too_old_builds" threshold
# This is needed to circumvent the limitations with dates of too basic databases (e.g. SQLite)
#df_known_builds = df_known_builds[df_known_builds.timestamp > too_old_builds]

In [ ]:
df_known_builds[df_known_builds.job=='osm-stage_3-merge/v9.0']

In [ ]:
query_robot_reports = f'''
SELECT main.timestamp, details.*
FROM {table_robot_reports} AS details
INNER JOIN {table_known_builds} AS main
ON details.job=main.job AND details.build=main.build
WHERE main.timestamp>DATETIME("{too_old_builds}")
ORDER BY details.job, details.build, details.starttime
'''

with engine.begin() as conn:
    df_all_build_reports = pd.read_sql(query_robot_reports, con=conn)

# Fixes some special data types
df_all_build_reports['timestamp'] = pd.to_datetime(df_all_build_reports.timestamp)
df_all_build_reports['job'] = df_all_build_reports.job.astype('category')
df_all_build_reports['id'] = df_all_build_reports.id.astype('category')
df_all_build_reports['name'] = df_all_build_reports.name.astype('category')
df_all_build_reports['source'] = df_all_build_reports.source.astype('category')
df_all_build_reports['starttime'] = pd.to_datetime(df_all_build_reports.starttime)
df_all_build_reports['endtime'] = pd.to_datetime(df_all_build_reports.endtime)
df_all_build_reports['status'] = df_all_build_reports.status.astype('category')
df_all_build_reports['failed_test_id'] = df_all_build_reports.failed_test_id.astype('category')
df_all_build_reports['failed_test_name'] = df_all_build_reports.failed_test_name.astype('category')
df_all_build_reports['failed_keyword'] = df_all_build_reports.failed_keyword.astype('category')

df_all_build_reports.tail()

In [ ]:
df_all_build_reports.info()

In [ ]:
query_robot_reports_extended = f'''
SELECT main.timestamp, details.*
FROM {table_robot_reports_extended} AS details
INNER JOIN {table_known_builds} AS main
ON details.job=main.job AND details.build=main.build
WHERE main.timestamp>DATETIME("{too_old_builds}")
ORDER BY details.job, details.build, details.starttime
'''

with engine.begin() as conn:
    df_all_build_reports_details = pd.read_sql(query_robot_reports_extended, con=conn)

# Fixes some special data types
df_all_build_reports_details['timestamp'] = pd.to_datetime(df_all_build_reports_details.timestamp)
df_all_build_reports_details['job'] = df_all_build_reports_details.job.astype('category')
df_all_build_reports_details['suite_id'] = df_all_build_reports_details.suite_id.astype('category')
df_all_build_reports_details['suite_name'] = df_all_build_reports_details.suite_name.astype('category')
df_all_build_reports_details['test_id'] = df_all_build_reports_details.test_id.astype('category')
df_all_build_reports_details['test_name'] = df_all_build_reports_details.test_name.astype('category')
df_all_build_reports_details['keyword_name'] = df_all_build_reports_details.keyword_name.astype('category')
df_all_build_reports_details['starttime'] = pd.to_datetime(df_all_build_reports_details.starttime)
df_all_build_reports_details['endtime'] = pd.to_datetime(df_all_build_reports_details.endtime)
df_all_build_reports_details['status'] = df_all_build_reports_details.status.astype('category')

df_all_build_reports_details

In [ ]:
df_all_build_reports_details.info()

## 3. Aggregated analysis of stability

In [ ]:
data = df_known_builds.copy()

# Calculates % of passed/failed sub-tests
data['pass_pct'] = data.pass_count / (data.pass_count + data.fail_count)
data['pass_pct'].fillna(0, inplace=True)
data['fail_pct'] = 1 - data.pass_pct
data['pass_pct'].fillna(100, inplace=True)

data

### 3.1 Finding sequences of successful builds and Robot reports

In [ ]:
data.build_result.value_counts()

In [ ]:
data.test_result.value_counts()

As seen in the cells right above, varios states are possible for the `build_result` and the `test_result` fields:

- `build_result` can be: `SUCCESS`, `FAILURE`, `UNSTABLE` or `ABORTED`.
- `test_result` can be: `FAIL`, `UNAVAILABLE` or `PASS`.

Based on these two states, 3 types of sequences of success/failure can be identified per build or test:

1. Successful builds/failed builds in a row: `grp_build_result`.
2. Successful test reports vs. test reports with fails in a row: `grp_test_result`.
3. Clean builds and tests vs. failures (of any kind) in a row: `grp_success_fail`.

For the identification of these sequences, we will need to check these states and values:

| Type of sequence   | Relevant state | OK sequence contains    | NOK sequence contains   | Ignore        |
|--------------------|----------------|-------------------------|-------------------------|---------------|
| `grp_build_result` | `build_result` | `SUCCESS` or `UNSTABLE` | `FAILURE`               | `ABORTED`     |
| `grp_test_result`  | `test_result`  | `PASS`                  | `FAIL`                  | `UNAVAILABLE` |
| `grp_success_fail` | `test_result`  | `PASS`                  | `FAIL` or `UNAVAILABLE` | N/A           |


Detects different grouping of segments:

In [ ]:
def find_sequence(column_state, test4different, ignored_value='DOES_NOT_EXIST'):
    # Determine which rows might lead to a change of sequence (i.e. those that won't be ignored)
    relevant_rows = (column_state!=ignored_value)

    # Creates an empty series with the same index as source series, "column_state"
    differences = pd.Series(index=column_state.index, dtype='boolean')

    # Tests if two consecutive relevant rows can be considered "different". If so, marks the row as "True"
    differences.loc[relevant_rows] = test4different(column_state.loc[relevant_rows], column_state.loc[relevant_rows].shift())

    # Irrelevant rows cannot lead to any change. Therefore, they will be marked as "False"
    differences.loc[~relevant_rows] = False

    # Each sequence will be given a different number as a result of a cumulative sum
    return differences.cumsum().astype('int64')

In [ ]:
# 1. Sequences for successful builds / failed builds in a row: `grp_build_result`
def test_grp_build_result(a, b):
    return (a!=b) & ((a=='FAILURE') | (b=='FAILURE'))

# 2. Successful test reports vs. test reports with fails in a row: `grp_test_result`
def test_grp_test_result(a, b):
    return (a!=b)

# 3. Clean builds and tests vs. failures (of any kind) in a row: `grp_success_fail`
def test_grp_success_fail(a, b):
    return (a!=b) & ((a=='PASS') | (b=='PASS'))

jobs = data.job.unique().tolist()
for job in jobs:
    job_data = data.loc[data.job==job, 'build_result']
    data.loc[data.job==job, 'grp_build_result'] = find_sequence(job_data, test_grp_build_result, ignored_value='ABORTED')
    job_data = data.loc[data.job==job, 'test_result']
    data.loc[data.job==job, 'grp_test_result'] = find_sequence(job_data, test_grp_test_result, ignored_value='UNAVAILABLE')
    data.loc[data.job==job, 'grp_success_fail'] = find_sequence(job_data, test_grp_success_fail)

data[data.job==job].tail()

Determines the length of sequences of success or failure:

In [ ]:
# Function that summarizes sequences of good results in a row (or with something wrong in a row)
def summarize_sequence(data, grouped_columns, aggregations_dict):
    sequence_summary = data.groupby(by=grouped_columns).agg(aggregations_dict).dropna().reset_index()
    
    # Flattens multi-level columns
    sequence_summary.columns = ['_'.join((col[1], col[0])) if col[1] else col[0] for col in sequence_summary.columns.values]

    return sequence_summary

In [ ]:
# Summarizes the 3 types of sequences in different tables

def agg_build_result(x):
    # If at least one in the sequence is 'FAILURE', the whole sequence is in failure
    test = (x=='FAILURE')
    return 'FAILURE' if test.sum() else 'SUCCESS'

def agg_test_result(x):
    # If at least one in the sequence is 'FAIL', the whole sequence is failing tests
    test = (x=='FAIL')
    return 'FAIL' if test.sum() else 'PASS'

def agg_success_fail(x):
    # If at least one in the sequence is 'PASS', the whole sequence is passing tests
    test = (x=='PASS')
    return 'PASS' if test.sum() else 'FAIL'

sequence_build_result = summarize_sequence(data, 
                            grouped_columns=['job', 'grp_build_result'],
                            aggregations_dict={'timestamp': ['min', 'max'], 'build': ['min', 'max'], 'build_result': agg_build_result}).rename(columns={agg_build_result.__name__+'_build_result': 'build_result'})
sequence_test_result = summarize_sequence(data, 
                            grouped_columns=['job', 'grp_test_result'],
                            aggregations_dict={'timestamp': ['min', 'max'], 'build': ['min', 'max'], 'test_result': agg_test_result}).rename(columns={agg_test_result.__name__+'_test_result': 'test_result'})
sequence_success_fail = summarize_sequence(data, 
                            grouped_columns=['job', 'grp_success_fail'],
                            aggregations_dict={'timestamp': ['min', 'max'], 'build': ['min', 'max'], 'test_result': agg_success_fail}).rename(columns={agg_success_fail.__name__+'_test_result': 'success_fail'})

In [ ]:
#sequence_build_result
#sequence_test_result
sequence_success_fail.tail()

Extends the length of each sequence up to the beginning of the next sequence, when applicable:

In [ ]:
def extend_sequence_length(min_timestamp, max_timestamp):
    # Assumes the entry series are already filtered by job
    df = pd.DataFrame({'min_timestamp': min_timestamp, 'next_timestamp': min_timestamp.shift(-1), 'max_timestamp': max_timestamp})
    return df.max(axis=1)

In [ ]:
for job in jobs:
    sequence_build_result.loc[sequence_build_result.job==job, 'max_timestamp'] = extend_sequence_length(
                                                                                sequence_build_result.loc[sequence_build_result.job==job, 'min_timestamp'], 
                                                                                sequence_build_result.loc[sequence_build_result.job==job, 'max_timestamp']
                                                                                )
    sequence_test_result.loc[sequence_test_result.job==job, 'max_timestamp'] = extend_sequence_length(
                                                                                sequence_test_result.loc[sequence_test_result.job==job, 'min_timestamp'], 
                                                                                sequence_test_result.loc[sequence_test_result.job==job, 'max_timestamp']
                                                                                )
    sequence_success_fail.loc[sequence_success_fail.job==job, 'max_timestamp'] = extend_sequence_length(
                                                                                sequence_success_fail.loc[sequence_success_fail.job==job, 'min_timestamp'], 
                                                                                sequence_success_fail.loc[sequence_success_fail.job==job, 'max_timestamp']
                                                                                )

In [ ]:
sequence_success_fail.tail()

### 3.2 Finding sequences of pass/fails per test suite

In [ ]:
test_suites_data = df_all_build_reports.copy().drop(columns=['source'])
test_suites_data

In [ ]:
# Detects grouping per sequence:
jobs = test_suites_data.job.unique().tolist()
for job in jobs:
    suite_ids = test_suites_data.loc[test_suites_data.job==job, 'id'].unique().tolist()
    for id in suite_ids:
        cond = (test_suites_data.job==job) & (test_suites_data.id==id)

        job_test_suites_data = test_suites_data.loc[cond, 'status']
        test_suites_data.loc[cond, 'grp_test_result'] = find_sequence(job_test_suites_data, test_grp_test_result, ignored_value='UNAVAILABLE')

test_suites_data[cond].tail()

In [ ]:
# Summarizes the sequences
sequence_test_suites = summarize_sequence(test_suites_data, 
                            grouped_columns=['job', 'id', 'name', 'grp_test_result'],
                            aggregations_dict={'timestamp': ['min', 'max'], 'build': ['min', 'max'], 'status': agg_test_result}).rename(columns={agg_test_result.__name__+'_status': 'test_result'})

#sequence_test_suites[(sequence_test_suites.job==job) & (sequence_test_suites.id==id)]
sequence_test_suites[(sequence_test_suites.job==job) & (sequence_test_suites.id=='s1-s32')]

In [ ]:
# Extends the sequence lenghts up to the beginning of the next sequence, when applicable
jobs = sequence_test_suites.job.unique().tolist()
for job in jobs:
    suite_ids = sequence_test_suites.loc[sequence_test_suites.job==job, 'id'].unique().tolist()
    for id in suite_ids:
        cond = (sequence_test_suites.job==job) & (sequence_test_suites.id==id)
        sequence_test_suites.loc[cond, 'max_timestamp'] = extend_sequence_length(
                                                          sequence_test_suites.loc[cond, 'min_timestamp'], 
                                                          sequence_test_suites.loc[cond, 'max_timestamp']
                                                          )
#sequence_test_suites[(sequence_test_suites.job==job) & (sequence_test_suites.id==id)]
sequence_test_suites[(sequence_test_suites.job==job) & (sequence_test_suites.id=='s1-s32')]

## 4. Reports

### 4.1 Aggregated success rate por test step

In [ ]:
plt.style.use('fivethirtyeight')

### 4.1.1 Rel NINE branch

In [ ]:
data_filtered = data[data.job=='osm-stage_3-merge/v9.0']

In [ ]:
def plot_aggregated_success_rate(data_filtered, title, filename=None):

    fig, ax = plt.subplots(figsize = (12,6))

    t = data_filtered.timestamp
    pass_pct = 100 * data_filtered.pass_pct
    fail_pct = 100 * data_filtered.fail_pct
    unavailable = (data_filtered.test_result=='UNAVAILABLE')*100

    ax.fill_between(t, fail_pct+pass_pct, pass_pct, color='red', alpha=0.5, label='Failed')
    ax.fill_between(t, pass_pct, color='lime', alpha=0.5, label='Passed')
    ax.fill_between(t, unavailable, color='dimgray', label='Unsuccessful builds')
    ax.axhline(100, color='black', linewidth=2, linestyle='--')

    ax.set_title(title, fontsize=16)
    ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))
    #ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (0.32, 0.4))
    fig.autofmt_xdate()

    fig.tight_layout()

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
plot_aggregated_success_rate(data_filtered,
    'Release NINE - % of successful test steps ('+ today + ')',
    os.path.join(outputs_folder, 'fully_successful_builds_v9'))

### 4.1.2 `master` branch

In [ ]:
data_filtered = data[data.job=='osm-stage_3-merge/master']

In [ ]:
plot_aggregated_success_rate(data_filtered,
    'Master branch - % of successful test steps ('+ today + ')',
    os.path.join(outputs_folder, 'fully_successful_builds_master'))

### 4.2 Analysis of builds and Robot tests

#### 4.2.1 Release NINE branch

##### 4.2.1.1 Success of Jenking builds

In [ ]:
def plot_aggregated_builds_and_tests(data_filtered, state_col, title, ok_states, nok_states, filename=None):

    fig, ax = plt.subplots(figsize = (12,6))

    for index, row in data_filtered.iterrows():
        #color = 'red' if row.build_result=='FAILURE' else 'lime'
        color = 'red' if row[state_col] in nok_states else 'lime'
        ax.axvspan(row.min_timestamp, row.max_timestamp, color=color, alpha=0.5)

    ax.set_title(title, fontsize=16)
    #ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))
    fig.autofmt_xdate()

    fig.tight_layout()

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
sequence_build_filtered = sequence_build_result[sequence_build_result.job=='osm-stage_3-merge/v9.0']
sequence_build_filtered.iloc[0:5]

In [ ]:
plot_aggregated_builds_and_tests(sequence_build_filtered, 'build_result',
    'Release NINE branch - build completions and failures ('+ today + ')',
    ok_states=['SUCCESS'], nok_states=['FAILURE'],
    filename=os.path.join(outputs_folder, 'successful_failed_builds_v9'))

In [ ]:
build_durations_filtered = sequence_build_filtered.copy()
build_durations_filtered['duration'] = build_durations_filtered.max_timestamp - build_durations_filtered.min_timestamp
#build_durations_filtered['duration'] = build_durations_filtered['duration'] / pd.to_timedelta(1, unit='D') # Expressed in days, allowing decimals
build_durations_filtered['duration'] = build_durations_filtered['duration'].dt.days

build_durations_filtered.drop(columns=['grp_build_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)
#build_durations_filtered.iloc[0:10]

builds_ok = build_durations_filtered.loc[build_durations_filtered.build_result=='SUCCESS', 'duration']
builds_nok = build_durations_filtered.loc[build_durations_filtered.build_result=='FAILURE', 'duration']

In [ ]:
def plot_density_builds_and_tests(data, title, labels, filename=None):

    fig, ax = plt.subplots(figsize = (12,6))

    n_bins = 25
    colors = ['green', 'red']
    #labels = ['OK', 'NOK']

    #ax.hist(data, n_bins, density=True, histtype='bar', color=colors, label=labels)
    #ax.hist(data, n_bins, density=True, histtype='bar', rwidth=0.8, label=labels, color=colors)
    ax.hist(data, n_bins, density=True, histtype='bar', rwidth=0.8, label=labels, color=colors, alpha=0.5)

    #ax.set_title(title, fontsize=20)
    fig.suptitle(title, fontsize=20)
    ax.set_xlabel('Duration of sequences (days)')
    ax.set_ylabel('Density')
    ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))

    fig.tight_layout()

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Rel NINE branch - Histogram of durations of build states (failure/success) ('+ today + ')',
                              labels=['SUCCESS', 'FAILURE'],
                              filename=os.path.join(outputs_folder, 'density_build_success_failure_v9'))

In [ ]:
def plot_kde_builds_and_tests(data, x, hue, colors, title, filename=None):

    fig, ax = plt.subplots(figsize = (12,6))

    sns.kdeplot(data=data,
                x=x, hue=hue,
                cut=0, bw_adjust=0.2,
                palette=colors, fill=True, alpha=0.4,
                ax=ax)

    fig.suptitle(title, fontsize=20)
    ax.set_xlabel('Duration of sequences (days)')

    if filename:
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
plot_kde_builds_and_tests(data=build_durations_filtered,
                          x='duration', hue='build_result', colors=['red', 'green'],
                          title='Rel NINE branch - KDE of durations of build states (failure/success) ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_build_success_failure_v9'))

##### 4.2.1.2 Success of Robot tests

In [ ]:
sequence_test_filtered = sequence_test_result[sequence_test_result.job=='osm-stage_3-merge/v9.0']
sequence_test_filtered.iloc[0:5]

In [ ]:
plot_aggregated_builds_and_tests(sequence_test_filtered, 'test_result',
    'Rel NINE branch - Robot tests status ('+ today + ')',
    ok_states=['PASS'], nok_states=['FAIL'],
    filename=os.path.join(outputs_folder, 'global_robot_status_v9'))

In [ ]:
test_durations_filtered = sequence_test_filtered.copy()
test_durations_filtered['duration'] = test_durations_filtered.max_timestamp - test_durations_filtered.min_timestamp
#test_durations_filtered['duration'] = test_durations_filtered['duration'] / pd.to_timedelta(1, unit='D') # Expressed in days, allowing decimals
test_durations_filtered['duration'] = test_durations_filtered['duration'].dt.days

test_durations_filtered.drop(columns=['grp_test_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)
#test_durations_filtered.iloc[0:10]

builds_ok = test_durations_filtered.loc[test_durations_filtered.test_result=='PASS', 'duration']
builds_nok = test_durations_filtered.loc[test_durations_filtered.test_result=='FAIL', 'duration']

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Rel NINE branch - Histogram of sequences of passed/failed Robot tests ('+ today + ')',
                              labels=['PASS', 'FAIL'],
                              filename=os.path.join(outputs_folder, 'density_robot_success_failure_v9'))

In [ ]:
plot_kde_builds_and_tests(data=test_durations_filtered,
                          x='duration', hue='test_result', colors=['green', 'red'],
                          title='Rel NINE branch - KDE of sequences of passed/failed Robot tests ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_robot_success_failure_v9'))

##### 4.2.1.3 Overall success (both build and Robot are ok)

In [ ]:
sequence_success_filtered = sequence_success_fail[sequence_success_fail.job=='osm-stage_3-merge/v9.0']
sequence_success_filtered.iloc[0:5]

In [ ]:
plot_aggregated_builds_and_tests(sequence_success_filtered, 'success_fail',
    'Rel NINE branch - Stability for point release ('+ today + ')',
    ok_states=['PASS'], nok_states=['FAIL'],
    filename=os.path.join(outputs_folder, 'global_stability_status_v9'))

In [ ]:
success_durations_filtered = sequence_success_filtered.copy()
success_durations_filtered['duration'] = success_durations_filtered.max_timestamp - success_durations_filtered.min_timestamp
#success_durations_filtered['duration'] = success_durations_filtered['duration'] / pd.to_timedelta(1, unit='D') # Expressed in days, allowing decimals
success_durations_filtered['duration'] = success_durations_filtered['duration'].dt.days

success_durations_filtered.drop(columns=['grp_success_fail', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)
#success_durations_filtered.iloc[0:10]

builds_ok = success_durations_filtered.loc[success_durations_filtered.success_fail=='PASS', 'duration']
builds_nok = success_durations_filtered.loc[success_durations_filtered.success_fail=='FAIL', 'duration']

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Rel NINE branch - Histogram of durations of global success ('+ today + ')',
                              labels=['SUCCESS', 'FAILURE'],
                              filename=os.path.join(outputs_folder, 'density_global_success_failure_v9'))

In [ ]:
plot_kde_builds_and_tests(data=success_durations_filtered,
                          x='duration', hue='success_fail', colors=['red', 'green'],
                          title='Rel NINE branch - KDE of durations of global success ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_global_success_failure_v9'))

##### 4.2.1.4 Builds completions and Robot status vs. Overall success (comparison)

In [ ]:
def plot_aggregated_stability_sequences(sequences, state_cols, titles, ok_states, nok_states, text=None, suptitle=None, filename=None, figsize=(14,8), tight=False):

    #fig, ax = plt.subplots(nrows=3, sharex=True, figsize = (14,8))
    fig, ax = plt.subplots(nrows=len(sequences), sharex=True, figsize=figsize)

    for i in range(len(sequences)):
        for index, row in sequences[i].iterrows():
            color = 'red' if row[state_cols[i]] in nok_states[i] else 'lime'
            ax[i].axvspan(row.min_timestamp, row.max_timestamp, color=color, alpha=0.5)
            if text:
                ax[i].text(0.5, 0.5, text[i], dict(size=14),
                           horizontalalignment='center', verticalalignment='center', transform=ax[i].transAxes, rasterized=False)

        if not text:
            ax[i].set_title(titles[i], fontsize=16)
        ax[i].set_yticklabels([])
        #ax[i].legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))

    fig.autofmt_xdate()

    if tight:
        fig.tight_layout()
    if suptitle:
        fig.suptitle(suptitle, fontsize=22)

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
plot_aggregated_stability_sequences(sequences=[sequence_build_filtered, sequence_test_filtered, sequence_success_filtered],
                                    state_cols=['build_result', 'test_result', 'success_fail'],
                                    titles=['Build completions and failures',
                                           'Robot tests status',
                                           'Stability for point release'],
                                    suptitle='Release NINE branch ('+ today + ')\n',
                                    ok_states=[['SUCCESS'], ['PASS'], ['PASS']],
                                    nok_states=[['FAILURE'], ['FAIL'], ['FAIL']],
                                    filename=os.path.join(outputs_folder, 'global_compared_stability_status_v9'))

##### 4.2.1.5 Sequences of pass/fails per test suites

In [ ]:
sequence_suites_filtered = sequence_test_suites[sequence_test_suites.job=='osm-stage_3-merge/v9.0']
sequence_suites_filtered.iloc[0:5]

In [ ]:
sequences = []
suites = sequence_suites_filtered.sort_values('name')['name'].unique().tolist()
for suite in suites:
    sequences.append(sequence_suites_filtered.loc[sequence_suites_filtered.name==suite])

sequences[4]

In [ ]:
n_suites = len(sequences)
state_cols = ['test_result'] * n_suites
ok_states = ['PASS'] * n_suites
nok_states = ['FAIL'] * n_suites
len(state_cols)

In [ ]:
text = suites
len(text)

In [ ]:
plot_aggregated_stability_sequences(sequences=sequences,
                                    state_cols=state_cols,
                                    titles= [''] * n_suites,
                                    text=text,
                                    suptitle='Release NINE branch ('+ today + ')',
                                    ok_states=ok_states,
                                    nok_states=nok_states,
                                    filename=os.path.join(outputs_folder, 'success_per_test_suite_status_v9'),
                                    figsize=(18,16),
                                    tight=False)

In [ ]:
sequence_suites_filtered.job.value_counts()

In [ ]:
duration_suites_filtered = sequence_suites_filtered.copy()
duration_suites_filtered['duration'] = duration_suites_filtered.max_timestamp - duration_suites_filtered.min_timestamp
#duration_suites_filtered.drop(columns=['grp_test_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)
duration_suites_filtered.drop(columns=['id', 'grp_test_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)
duration_suites_filtered

In [ ]:
#suites_success = duration_suites_filtered.groupby(by=['job', 'id', 'name', 'test_result']).sum().reset_index()
suites_success = duration_suites_filtered.groupby(by=['job', 'name', 'test_result']).sum().reset_index()
suites_success.duration = pd.to_timedelta(suites_success.duration)
suites_success = suites_success[suites_success.job=='osm-stage_3-merge/v9.0']
#suites_success = suites_success.set_index(['job', 'id', 'name', 'test_result'])
suites_success = suites_success.set_index(['job', 'name', 'test_result'])
#suites_success.columns.droplevel(level=-1)

In [ ]:
suites_success

In [ ]:
#fail_pass_durations_per_suite = suites_success.unstack().reset_index().drop(columns=['job'])
fail_pass_durations_per_suite = suites_success.unstack().reset_index()
fail_pass_durations_per_suite.columns = ['_'.join(col) if col[1] else col[0] for col in fail_pass_durations_per_suite.columns]
fail_pass_durations_per_suite.drop(columns=['job'], inplace=True)
# fail_pass_durations_per_suite['duration_FAIL'] = fail_pass_durations_per_suite['duration_FAIL'] / pd.Timedelta(days=1)
# fail_pass_durations_per_suite['duration_PASS'] = fail_pass_durations_per_suite['duration_PASS'] / pd.Timedelta(days=1)
fail_pass_durations_per_suite['duration_FAIL'] = fail_pass_durations_per_suite['duration_FAIL'].dt.days
fail_pass_durations_per_suite['duration_PASS'] = fail_pass_durations_per_suite['duration_PASS'].dt.days

#fail_pass_durations_per_suite = fail_pass_durations_per_suite.sort_values(by='duration_FAIL', ascending=False)
fail_pass_durations_per_suite = fail_pass_durations_per_suite.sort_values(by='duration_FAIL')
fail_pass_durations_per_suite

In [ ]:
def plot_days_suites_ok_nok(suites, days_failing, days_passing, title, filename=None):

    fig, ax = plt.subplots(figsize = (12,16))

    plt.barh(suites, days_failing, color='red', alpha=0.5, label='Failing')
    plt.barh(suites, days_passing, color='lime', alpha=0.5, left=days_failing, label='Passing')

    #ax.set_title(title, fontsize=20)
    fig.suptitle(title, fontsize=20)
    ax.set_xlabel('Number of days')
    #ax.legend(fontsize=12, fancybox=True, shadow=True, borderpad=1, bbox_to_anchor = (1, 1))

    fig.tight_layout()

    if (filename):
        fig.savefig(filename + '.png', dpi=300)
        fig.savefig(filename + '.svg')

    plt.show()

In [ ]:
plot_days_suites_ok_nok(suites=fail_pass_durations_per_suite.name,
                        days_failing=fail_pass_durations_per_suite.duration_FAIL,
                        days_passing=fail_pass_durations_per_suite.duration_PASS,
                        title='Release NINE - Failing days per test suite ('+ today + ')',
                        filename=os.path.join(outputs_folder, 'failing_days_per_suite_v9'))

#### 4.2.2 Master branch

In [ ]:
reference_job = 'osm-stage_3-merge/master'

##### 4.2.2.1 Success of Jenking builds

In [ ]:
sequence_build_filtered = sequence_build_result[sequence_build_result.job==reference_job]

In [ ]:
plot_aggregated_builds_and_tests(sequence_build_filtered, 'build_result',
    'Master branch - build completions and failures ('+ today + ')',
    ok_states=['SUCCESS'], nok_states=['FAILURE'],
    filename=os.path.join(outputs_folder, 'successful_failed_builds_master'))

In [ ]:
build_durations_filtered = sequence_build_filtered.copy()
build_durations_filtered['duration'] = build_durations_filtered.max_timestamp - build_durations_filtered.min_timestamp
build_durations_filtered['duration'] = build_durations_filtered['duration'].dt.days

build_durations_filtered.drop(columns=['grp_build_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

builds_ok = build_durations_filtered.loc[build_durations_filtered.build_result=='SUCCESS', 'duration']
builds_nok = build_durations_filtered.loc[build_durations_filtered.build_result=='FAILURE', 'duration']

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Master branch - Histogram of durations of build states (failure/success) ('+ today + ')',
                              labels=['SUCCESS', 'FAILURE'],
                              filename=os.path.join(outputs_folder, 'density_build_success_failure_master'))

In [ ]:
plot_kde_builds_and_tests(data=build_durations_filtered,
                          x='duration', hue='build_result', colors=['red', 'green'],
                          title='Master branch - KDE of durations of build states (failure/success) ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_build_success_failure_master'))

##### 4.2.2.2 Success of Robot tests

In [ ]:
sequence_test_filtered = sequence_test_result[sequence_test_result.job==reference_job]
sequence_test_filtered.iloc[0:5]

In [ ]:
plot_aggregated_builds_and_tests(sequence_test_filtered, 'test_result',
    'Master branch - Robot tests status ('+ today + ')',
    ok_states=['PASS'], nok_states=['FAIL'],
    filename=os.path.join(outputs_folder, 'global_robot_status_master'))

In [ ]:
test_durations_filtered = sequence_test_filtered.copy()
test_durations_filtered['duration'] = test_durations_filtered.max_timestamp - test_durations_filtered.min_timestamp
test_durations_filtered['duration'] = test_durations_filtered['duration'].dt.days

test_durations_filtered.drop(columns=['grp_test_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

builds_ok = test_durations_filtered.loc[test_durations_filtered.test_result=='PASS', 'duration']
builds_nok = test_durations_filtered.loc[test_durations_filtered.test_result=='FAIL', 'duration']

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Master branch - Histogram of sequences of passed/failed Robot tests ('+ today + ')',
                              labels=['PASS', 'FAIL'],
                              filename=os.path.join(outputs_folder, 'density_robot_success_failure_master'))

In [ ]:
plot_kde_builds_and_tests(data=test_durations_filtered,
                          x='duration', hue='test_result', colors=['green', 'red'],
                          title='Master branch - KDE of sequences of passed/failed Robot tests ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_robot_success_failure_master'))

##### 4.2.2.3 Overall success (both build and Robot are ok)

In [ ]:
sequence_success_filtered = sequence_success_fail[sequence_success_fail.job==reference_job]

In [ ]:
plot_aggregated_builds_and_tests(sequence_success_filtered, 'success_fail',
    'Master branch - Stability for point release ('+ today + ')',
    ok_states=['PASS'], nok_states=['FAIL'],
    filename=os.path.join(outputs_folder, 'global_stability_status_master'))

In [ ]:
success_durations_filtered = sequence_success_filtered.copy()
success_durations_filtered['duration'] = success_durations_filtered.max_timestamp - success_durations_filtered.min_timestamp
success_durations_filtered['duration'] = success_durations_filtered['duration'].dt.days

success_durations_filtered.drop(columns=['grp_success_fail', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

builds_ok = success_durations_filtered.loc[success_durations_filtered.success_fail=='PASS', 'duration']
builds_nok = success_durations_filtered.loc[success_durations_filtered.success_fail=='FAIL', 'duration']

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Master branch - Histogram of durations of global success ('+ today + ')',
                              labels=['SUCCESS', 'FAILURE'],
                              filename=os.path.join(outputs_folder, 'density_global_success_failure_master'))

In [ ]:
plot_kde_builds_and_tests(data=success_durations_filtered,
                          x='duration', hue='success_fail', colors=['red', 'green'],
                          title='Master branch - KDE of durations of global success ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_global_success_failure_master'))

##### 4.2.2.4 Builds completions and Robot status vs. Overall success (comparison)

In [ ]:
plot_aggregated_stability_sequences(sequences=[sequence_build_filtered, sequence_test_filtered, sequence_success_filtered],
                                    state_cols=['build_result', 'test_result', 'success_fail'],
                                    titles=['Build completions and failures',
                                           'Robot tests status',
                                           'Stability for point release'],
                                    suptitle='Master branch ('+ today + ')\n',
                                    ok_states=[['SUCCESS'], ['PASS'], ['PASS']],
                                    nok_states=[['FAILURE'], ['FAIL'], ['FAIL']],
                                    filename=os.path.join(outputs_folder, 'global_compared_stability_status_master'))

##### 4.2.2.5 Sequences of pass/fails per test suites

In [ ]:
sequence_suites_filtered = sequence_test_suites[sequence_test_suites.job==reference_job]

In [ ]:
sequences = []
suites = sequence_suites_filtered.sort_values('name')['name'].unique().tolist()
for suite in suites:
    sequences.append(sequence_suites_filtered.loc[sequence_suites_filtered.name==suite])

In [ ]:
text = suites
n_suites = len(sequences)
state_cols = ['test_result'] * n_suites
ok_states = ['PASS'] * n_suites
nok_states = ['FAIL'] * n_suites

In [ ]:
plot_aggregated_stability_sequences(sequences=sequences,
                                    state_cols=state_cols,
                                    titles= [''] * n_suites,
                                    text=text,
                                    suptitle='Master branch ('+ today + ')',
                                    ok_states=ok_states,
                                    nok_states=nok_states,
                                    filename=os.path.join(outputs_folder, 'success_per_test_suite_status_master'),
                                    figsize=(18,16),
                                    tight=False)

In [ ]:
duration_suites_filtered = sequence_suites_filtered.copy()
duration_suites_filtered['duration'] = duration_suites_filtered.max_timestamp - duration_suites_filtered.min_timestamp
duration_suites_filtered.drop(columns=['id', 'grp_test_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

In [ ]:
suites_success = duration_suites_filtered.groupby(by=['job', 'name', 'test_result']).sum().reset_index()
suites_success.duration = pd.to_timedelta(suites_success.duration)
suites_success = suites_success[suites_success.job==reference_job]
suites_success = suites_success.set_index(['job', 'name', 'test_result'])

In [ ]:
fail_pass_durations_per_suite = suites_success.unstack().reset_index()
fail_pass_durations_per_suite.columns = ['_'.join(col) if col[1] else col[0] for col in fail_pass_durations_per_suite.columns]
fail_pass_durations_per_suite.drop(columns=['job'], inplace=True)
fail_pass_durations_per_suite['duration_FAIL'] = fail_pass_durations_per_suite['duration_FAIL'].dt.days
fail_pass_durations_per_suite['duration_PASS'] = fail_pass_durations_per_suite['duration_PASS'].dt.days

fail_pass_durations_per_suite = fail_pass_durations_per_suite.sort_values(by='duration_FAIL')

In [ ]:
plot_days_suites_ok_nok(suites=fail_pass_durations_per_suite.name,
                        days_failing=fail_pass_durations_per_suite.duration_FAIL,
                        days_passing=fail_pass_durations_per_suite.duration_PASS,
                        title='Master branch - Failing days per test suite ('+ today + ')',
                        filename=os.path.join(outputs_folder, 'failing_days_per_suite_master'))

#### 4.2.3 Release TEN branch

In [ ]:
reference_job = 'osm-stage_3-merge/v10.0'

##### 4.2.3.1 Success of Jenking builds

In [ ]:
sequence_build_filtered = sequence_build_result[sequence_build_result.job==reference_job]

In [ ]:
sequence_build_filtered.tail()

In [ ]:
plot_aggregated_builds_and_tests(sequence_build_filtered, 'build_result',
    'Release TEN branch - build completions and failures ('+ today + ')',
    ok_states=['SUCCESS'], nok_states=['FAILURE'],
    filename=os.path.join(outputs_folder, 'successful_failed_builds_v10'))

In [ ]:
build_durations_filtered = sequence_build_filtered.copy()
build_durations_filtered['duration'] = build_durations_filtered.max_timestamp - build_durations_filtered.min_timestamp
build_durations_filtered['duration'] = build_durations_filtered['duration'].dt.days

build_durations_filtered.drop(columns=['grp_build_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

builds_ok = build_durations_filtered.loc[build_durations_filtered.build_result=='SUCCESS', 'duration']
builds_nok = build_durations_filtered.loc[build_durations_filtered.build_result=='FAILURE', 'duration']

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Release TEN branch - Histogram of durations of build states (failure/success) ('+ today + ')',
                              labels=['SUCCESS', 'FAILURE'],
                              filename=os.path.join(outputs_folder, 'density_build_success_failure_v10'))

In [ ]:
plot_kde_builds_and_tests(data=build_durations_filtered,
                          x='duration', hue='build_result', colors=['red', 'green'],
                          title='Release TEN branch - KDE of durations of build states (failure/success) ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_build_success_failure_v10'))

##### 4.2.3.2 Success of Robot tests

In [ ]:
sequence_test_filtered = sequence_test_result[sequence_test_result.job==reference_job]
sequence_test_filtered.iloc[0:5]

In [ ]:
plot_aggregated_builds_and_tests(sequence_test_filtered, 'test_result',
    'Release TEN branch - Robot tests status ('+ today + ')',
    ok_states=['PASS'], nok_states=['FAIL'],
    filename=os.path.join(outputs_folder, 'global_robot_status_v10'))

In [ ]:
test_durations_filtered = sequence_test_filtered.copy()
test_durations_filtered['duration'] = test_durations_filtered.max_timestamp - test_durations_filtered.min_timestamp
test_durations_filtered['duration'] = test_durations_filtered['duration'].dt.days

test_durations_filtered.drop(columns=['grp_test_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

builds_ok = test_durations_filtered.loc[test_durations_filtered.test_result=='PASS', 'duration']
builds_nok = test_durations_filtered.loc[test_durations_filtered.test_result=='FAIL', 'duration']

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Release TEN branch - Histogram of sequences of passed/failed Robot tests ('+ today + ')',
                              labels=['PASS', 'FAIL'],
                              filename=os.path.join(outputs_folder, 'density_robot_success_failure_v10'))

In [ ]:
plot_kde_builds_and_tests(data=test_durations_filtered,
                          x='duration', hue='test_result', colors=['green', 'red'],
                          title='Release TEN branch - KDE of sequences of passed/failed Robot tests ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_robot_success_failure_v10'))

##### 4.2.3.3 Overall success (both build and Robot are ok)

In [ ]:
sequence_success_filtered = sequence_success_fail[sequence_success_fail.job==reference_job]

In [ ]:
plot_aggregated_builds_and_tests(sequence_success_filtered, 'success_fail',
    'Release TEN branch - Stability for point release ('+ today + ')',
    ok_states=['PASS'], nok_states=['FAIL'],
    filename=os.path.join(outputs_folder, 'global_stability_status_v10'))

In [ ]:
success_durations_filtered = sequence_success_filtered.copy()
success_durations_filtered['duration'] = success_durations_filtered.max_timestamp - success_durations_filtered.min_timestamp
success_durations_filtered['duration'] = success_durations_filtered['duration'].dt.days

success_durations_filtered.drop(columns=['grp_success_fail', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

builds_ok = success_durations_filtered.loc[success_durations_filtered.success_fail=='PASS', 'duration']
builds_nok = success_durations_filtered.loc[success_durations_filtered.success_fail=='FAIL', 'duration']

In [ ]:
plot_density_builds_and_tests(data=[builds_ok, builds_nok],
                              title='Release TEN branch - Histogram of durations of global success ('+ today + ')',
                              labels=['SUCCESS', 'FAILURE'],
                              filename=os.path.join(outputs_folder, 'density_global_success_failure_v10'))

In [ ]:
plot_kde_builds_and_tests(data=success_durations_filtered,
                          x='duration', hue='success_fail', colors=['red', 'green'],
                          title='Release TEN branch - KDE of durations of global success ('+ today + ')',
                          filename=os.path.join(outputs_folder, 'kde_global_success_failure_v10'))

##### 4.2.3.4 Builds completions and Robot status vs. Overall success (comparison)

In [ ]:
# plot_aggregated_stability_sequences(sequences=[sequence_build_result, sequence_test_result, sequence_success_fail],
#                                     state_cols=['build_result', 'test_result', 'success_fail'],
#                                     titles=['Build completions and failures',
#                                            'Robot tests status',
#                                            'Stability for point release'],
#                                     suptitle='Release TEN branch\n',
#                                     ok_states=[['SUCCESS'], ['PASS'], ['PASS']],
#                                     nok_states=[['FAILURE'], ['FAIL'], ['FAIL']],
#                                     filename=os.path.join(outputs_folder, 'global_compared_stability_status_v10'))

In [ ]:
plot_aggregated_stability_sequences(sequences=[sequence_build_filtered, sequence_test_filtered, sequence_success_filtered],
                                    state_cols=['build_result', 'test_result', 'success_fail'],
                                    titles=['Build completions and failures',
                                           'Robot tests status',
                                           'Stability for point release'],
                                    suptitle='Release TEN branch ('+ today + ')\n',
                                    ok_states=[['SUCCESS'], ['PASS'], ['PASS']],
                                    nok_states=[['FAILURE'], ['FAIL'], ['FAIL']],
                                    filename=os.path.join(outputs_folder, 'global_compared_stability_status_v10'))

##### 4.2.3.5 Sequences of pass/fails per test suites

In [ ]:
sequence_suites_filtered = sequence_test_suites[sequence_test_suites.job==reference_job]

In [ ]:
sequences = []
suites = sequence_suites_filtered.sort_values('name')['name'].unique().tolist()
for suite in suites:
    sequences.append(sequence_suites_filtered.loc[sequence_suites_filtered.name==suite])

In [ ]:
text = suites
n_suites = len(sequences)
state_cols = ['test_result'] * n_suites
ok_states = ['PASS'] * n_suites
nok_states = ['FAIL'] * n_suites

In [ ]:
plot_aggregated_stability_sequences(sequences=sequences,
                                    state_cols=state_cols,
                                    titles= [''] * n_suites,
                                    text=text,
                                    suptitle='Release TEN branch ('+ today + ')',
                                    ok_states=ok_states,
                                    nok_states=nok_states,
                                    filename=os.path.join(outputs_folder, 'success_per_test_suite_status_v10'),
                                    figsize=(18,16),
                                    tight=False)

In [ ]:
duration_suites_filtered = sequence_suites_filtered.copy()
duration_suites_filtered['duration'] = duration_suites_filtered.max_timestamp - duration_suites_filtered.min_timestamp
duration_suites_filtered.drop(columns=['id', 'grp_test_result', 'min_timestamp', 'max_timestamp', 'min_build', 'max_build'], inplace=True)

In [ ]:
suites_success = duration_suites_filtered.groupby(by=['job', 'name', 'test_result']).sum().reset_index()
suites_success.duration = pd.to_timedelta(suites_success.duration)
suites_success = suites_success[suites_success.job==reference_job]
suites_success = suites_success.set_index(['job', 'name', 'test_result'])

In [ ]:
fail_pass_durations_per_suite = suites_success.unstack().reset_index()
fail_pass_durations_per_suite.columns = ['_'.join(col) if col[1] else col[0] for col in fail_pass_durations_per_suite.columns]
fail_pass_durations_per_suite.drop(columns=['job'], inplace=True)
fail_pass_durations_per_suite['duration_FAIL'] = fail_pass_durations_per_suite['duration_FAIL'].dt.days
fail_pass_durations_per_suite['duration_PASS'] = fail_pass_durations_per_suite['duration_PASS'].dt.days

fail_pass_durations_per_suite = fail_pass_durations_per_suite.sort_values(by='duration_FAIL')

In [ ]:
plot_days_suites_ok_nok(suites=fail_pass_durations_per_suite.name,
                        days_failing=fail_pass_durations_per_suite.duration_FAIL,
                        days_passing=fail_pass_durations_per_suite.duration_PASS,
                        title='Release TEN branch - Failing days per test suite ('+ today + ')',
                        filename=os.path.join(outputs_folder, 'failing_days_per_suite_v10'))